In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
def process_damage_log(log_line):
    springs, contig = log_line.split()
    contig = [ int(i) for i in contig.split(',')]
    unknown = []
    for i,c in enumerate(springs):
        if c == '?':
            unknown.append(i)
    return springs, unknown, contig

In [3]:
def calculate_continguous(spring):
    groups=[]
    in_group=False
    cnt=0
    for c in spring:
        if c == '#' and not in_group:
            cnt=1
            in_group =True
        elif c == '#' and in_group:
            cnt+=1
        elif c == '.' and in_group:
            groups.append(cnt)
            cnt=0
            in_group =False
    if cnt >0:
        groups.append(cnt)
    return groups

In [4]:
from itertools import combinations

def produce_trial_configs(springs, unknown, contig):
    n_needed = sum(contig) - sum([ i=='#' for i in springs])
    valid_ways=0
    for switch in combinations(unknown, n_needed):
        trial_string = springs
        for c in switch:
            trial_string = trial_string[:c]+'#'+trial_string[c+1:]
        trial_string = trial_string.replace('?','.')
#        print(trial_string, calculate_continguous(trial_string))
        if tuple(contig) == tuple(calculate_continguous(trial_string)):
            valid_ways+=1
        
    return valid_ways

In [5]:
test='???.### 1,1,3'
spring, unk, contig=process_damage_log(test)
produce_trial_configs(spring, unk, contig)

1

In [6]:
def do_part_one(file_path):
    
    with open(file_path,'r') as f:
        spring_log = [ line.strip() for line in f.readlines()]

    ways=0
    for proc_line in spring_log:
        spring, unk, contig = process_damage_log(proc_line)
        cur_ways = produce_trial_configs(spring, unk, contig)
#        print(spring, unk, contig,cur_ways)
        ways+=cur_ways
        
    print(ways)

In [7]:
%%time
do_part_one('input_data/day_12.txt')

7653
CPU times: user 1.55 s, sys: 1.9 ms, total: 1.55 s
Wall time: 1.56 s


# Part 2

In [68]:
from functools import cache
def check_if_block(spring_data, blocks):
    size = blocks[0]
    obs = sum([ c !='.' for c in spring_data[:size] ])
    if len(spring_data) == size:
        return (obs == size)
    return (obs == size) and ( spring_data[size] != '#')

@cache
def trial_configs_recursive(springs, contig):
    
    total_needed = sum(contig) 
    known = sum([ c=='#' for c in springs])
    possible = sum([ c!='.' for c in springs])
    
    if total_needed <  known or possible < total_needed:
        return 0 
    if total_needed ==0 or len(contig) == 0:
        return 1
    
    if springs[0] == '.': 
        return trial_configs_recursive(springs[1:], contig)
    
    if springs[0] == '#': 
        if check_if_block( springs, contig):
            size = contig[0]
            if len(springs) == size:
                return 1 
            else:
                return trial_configs_recursive(springs[size+1:], tuple(contig[1:]))
        else:
            return 0
        
    new_spring = '#'+springs[1:]
    return trial_configs_recursive(springs[1:],contig) + trial_configs_recursive(new_spring,contig)
    

def do_part_two(file_path):
    
    with open(file_path,'r') as f:
        spring_log = [ line.strip() for line in f.readlines()]

    ways=0
    for proc_line in spring_log:
        springs, unk, contig = process_damage_log(proc_line)
        
        springs = '?'.join([springs]*5)
        contig = tuple((contig*5).copy())
        
        springs = springs.strip('.')
        cur_ways = trial_configs_recursive(springs,contig)
        ways+= cur_ways
        
    print('\n',ways)

In [69]:
%%time
do_part_two('input_data/day_12.txt')


 60681419004564
CPU times: user 962 ms, sys: 9.1 ms, total: 971 ms
Wall time: 973 ms
